In [339]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# from imblearn.over_sampling import SMOTE
from konlpy.tag import  Okt
from sklearn.cluster import DBSCAN
okt=Okt()

ImportError: cannot import name 'DistanceMetric' from 'sklearn.metrics' (C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\__init__.py)

In [327]:
naver=pd.read_csv('네이버페이_전처리.csv')
kakao=pd.read_csv('카카오페이_전처리.csv')
samsung=pd.read_csv('삼성페이_전처리.csv')

In [328]:
kakao=kakao.dropna()

In [329]:
#필요한 행 추출
naver=naver[['STAR','cleaning_REVIEW']]
kakao=kakao[['STAR','cleaning_REVIEW']]
samsung=samsung[['STAR','cleaning_REVIEW']]

In [330]:
def pos_filtering(text): 
    pos_word_list = okt.pos(text, stem = True) #원형으로
    pos_list = ['Noun'] #명사 추출
    pos_filtered_word_list = []

    for word, pos in pos_word_list: # 품사필터링
        if pos in pos_list:
            pos_filtered_word_list.append(word)

    return pos_filtered_word_list

In [331]:
#명사토큰화
kakao['token_REVIEW']=kakao.cleaning_REVIEW.map(lambda x : pos_filtering(x))
samsung['token_REVIEW']=samsung.cleaning_REVIEW.map(lambda x : pos_filtering(x))
naver['token_REVIEW']=naver.cleaning_REVIEW.map(lambda x : pos_filtering(x))

#한글자 지우기
kakao['token_REVIEW'] = kakao['token_REVIEW'].map(lambda x :[word for word in x if len(word) != 1])
samsung['token_REVIEW'] = samsung['token_REVIEW'].map(lambda x :[word for word in x if len(word) != 1])
naver['token_REVIEW'] = naver['token_REVIEW'].map(lambda x :[word for word in x if len(word) != 1])


In [332]:
#감성 라벨링 (1~3 => 0; 4~5 => 1)
naver.STAR=naver.STAR.apply(lambda x : 1 if x>3 else 0)
kakao.STAR=kakao.STAR.apply(lambda x : 1 if x>3 else 0)
samsung.STAR=samsung.STAR.apply(lambda x : 1 if x>3 else 0)

In [333]:
#리스트빈것지우기
naver['token_REVIEW']=naver['token_REVIEW'].apply( lambda x : "" if x==[] else x)
samsung['token_REVIEW']=samsung['token_REVIEW'].apply( lambda x : "" if x==[] else x)
kakao['token_REVIEW']=kakao['token_REVIEW'].apply( lambda x : "" if x==[] else x)
naver=naver[naver.token_REVIEW!=""]
samsung=samsung[samsung.token_REVIEW!=""]
kakao=kakao[kakao.token_REVIEW!=""]
naver.reset_index(drop=True,inplace=True)
samsung.reset_index(drop=True,inplace=True)
kakao.reset_index(drop=True,inplace=True)

In [334]:
# #str => 리스트 타입 변환
# naver.token_REVIEW=naver.token_REVIEW.apply(lambda x: eval("".join(x)))
# samsung.token_REVIEW=samsung.token_REVIEW.apply(lambda x: eval("".join(x)))
# kakao.token_REVIEW=kakao.token_REVIEW.apply(lambda x: eval("".join(x)))

In [335]:
naver.token_REVIEW=naver.token_REVIEW.apply(lambda x: " ".join(x))
samsung.token_REVIEW=samsung.token_REVIEW.apply(lambda x: " ".join(x))
kakao.token_REVIEW=kakao.token_REVIEW.apply(lambda x: " ".join(x))

## 네이버 긍부정단어 예측 

In [297]:
naver.STAR.value_counts()

1    309
0    267
Name: STAR, dtype: int64

In [280]:
#데이터 스플릿 (네이버는 양이 적어 8:2로 분리)
train_texts, test_texts, train_labels, test_labels = train_test_split(naver.token_REVIEW, naver.STAR, test_size=0.2, random_state=123)

In [281]:
#TF-IDF
vectorizer = TfidfVectorizer(min_df=1, norm='l2', ngram_range=(1,1))
train_features = vectorizer.fit_transform(train_texts)
#최적모델찾기
clf = LogisticRegression(random_state=0)
params = {"C":[0.1,0.5,1,5,10]}
model= GridSearchCV(clf, param_grid=params, cv = 5, scoring='accuracy') #그리드서치로 돌리면 계수안나옴
model.fit(train_features, train_labels)
print('최적 하이퍼파라미터 :{0}, 정확도 :{1:.3f}'.format(model.best_params_,model.best_score_))

최적 하이퍼파라미터 :{'C': 5}, 정확도 :0.696


In [282]:
model = LogisticRegression(C=5)

In [284]:
model.fit(train_features, train_labels)
test_features = vectorizer.transform(test_texts)
pred_labels = model.predict(test_features)
print('잘못 분류된 강의평: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

잘못 분류된 강의평: 28 out of 116
Accuracy: 0.76


In [285]:
coefficients = model.coef_.tolist()
sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
vocablist = [word for word, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]
#긍부정단어
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

추가 (3.035)
워치 (2.837)
자산 (2.433)
애플 (2.369)
연동 (2.308)
정말 (2.215)
자주 (1.834)
아주 (1.792)
편의점 (1.679)
그동안 (1.630)
쓰기 (1.527)
사용 (1.520)
별도 (1.465)
결제 (1.407)
올해 (1.396)
강추 (1.391)
얼룬해주세욤 (1.379)
재미 (1.331)
제일 (1.323)
방식 (1.322)
통신 (1.321)
플레이스토어 (1.309)
비교 (1.293)
소액 (1.281)
무료 (1.253)
버그 (1.235)
평소 (1.234)
대박 (1.232)
배송 (1.230)
위젯 (1.229)
이미지 (1.220)
디자인 (1.210)
기대 (1.207)
알뜰 (1.162)
주시 (1.159)
공감 (1.159)
굿굿 (1.159)
굿굿굿 (1.159)
기분 (1.159)
따끈따끈 (1.159)
아침 (1.159)
원조 (1.159)
제목 (1.159)
캐뱅추갖 (1.159)
한지 (1.159)
네퍼 (1.141)
저번 (1.140)
포인트 (1.132)
최고 (1.124)
멤버십 (1.077)
문의 (-1.127)
제대로 (-1.133)
요청 (-1.139)
신용 (-1.154)
다이소 (-1.160)
개선 (-1.182)
거래 (-1.192)
게이 (-1.196)
금액 (-1.206)
장바구니 (-1.212)
어찌 (-1.216)
다시 (-1.219)
업데이트 (-1.232)
어플 (-1.247)
씨유 (-1.271)
시작 (-1.277)
이상 (-1.293)
위치 (-1.293)
로뱅 (-1.297)
가입 (-1.354)
반복 (-1.376)
뭔가 (-1.430)
다운로드 (-1.436)
삭제 (-1.502)
투명 (-1.534)
이유 (-1.542)
아이디 (-1.566)
정보 (-1.573)
한번 (-1.588)
은행 (-1.596)
편의 (-1.613)
고객 (-1.763)
로그인 (-1.789)
주심 (-1.804)
인식 (-1.846)
문

## 카카오 긍부정 단어 예측

In [295]:
kakao.STAR.value_counts()

0    1733
1    1276
Name: STAR, dtype: int64

In [288]:
#데이터 스플릿 
train_texts, test_texts, train_labels, test_labels = train_test_split(kakao.token_REVIEW, kakao.STAR, test_size=0.3, random_state=123)

In [289]:
#TF-IDF
vectorizer = TfidfVectorizer(min_df=1, norm='l2', ngram_range=(1,1))
train_features = vectorizer.fit_transform(train_texts)
#최적모델찾기
clf = LogisticRegression(random_state=0)
params = {"C":[0.1,0.5,1,5,10]}
model= GridSearchCV(clf, param_grid=params, cv = 5, scoring='accuracy') 
model.fit(train_features, train_labels)
print('최적 하이퍼파라미터 :{0}, 정확도 :{1:.3f}'.format(model.best_params_,model.best_score_))

최적 하이퍼파라미터 :{'C': 1}, 정확도 :0.777


In [291]:
model = LogisticRegression(C=1)

In [292]:
model.fit(train_features, train_labels)
test_features = vectorizer.transform(test_texts)
pred_labels = model.predict(test_features)
print('잘못 분류된 강의평: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

잘못 분류된 강의평: 190 out of 903
Accuracy: 0.79


In [293]:
coefficients = model.coef_.tolist()
sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
vocablist = [word for word, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]
#긍부정단어
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

당첨 (2.544)
최고 (2.493)
아주 (2.443)
추가 (1.876)
굿굿 (1.865)
감사 (1.789)
대박 (1.749)
이벤트 (1.720)
정말 (1.699)
사용 (1.313)
기분 (1.285)
조금 (1.242)
와우 (1.236)
행운 (1.205)
쓰기 (1.185)
럭키 (1.133)
재미 (1.109)
실용 (0.995)
기대 (0.956)
이번 (0.924)
모든 (0.871)
항상 (0.868)
무지 (0.865)
원조 (0.865)
자주 (0.863)
추천 (0.839)
단위 (0.832)
만족 (0.776)
변경 (0.773)
세상 (0.764)
신속 (0.764)
저번 (0.744)
가지 (0.717)
금융 (0.697)
보기 (0.692)
이용 (0.683)
재부팅 (0.681)
굿굿굿 (0.677)
생활 (0.667)
활용 (0.663)
오당 (0.663)
이벤 (0.663)
체고 (0.663)
헤헤 (0.663)
난리 (0.649)
미국 (0.624)
득템 (0.622)
도움 (0.622)
무료 (0.610)
혜택 (0.593)
회사 (-0.885)
이상 (-0.924)
토스 (-0.925)
고객 (-0.926)
환불 (-0.949)
인증서 (-0.955)
페미 (-0.957)
다른 (-0.959)
설치 (-0.969)
대체 (-0.996)
하나요 (-0.999)
조회 (-1.011)
다운로드 (-1.046)
카톡 (-1.046)
촬영 (-1.052)
증권 (-1.085)
에러 (-1.086)
이후 (-1.093)
먹통 (-1.094)
문제 (-1.103)
거지 (-1.106)
실행 (-1.112)
비밀번호 (-1.117)
이유 (-1.119)
하라 (-1.121)
제대로 (-1.127)
도대체 (-1.145)
입력 (-1.180)
출금 (-1.234)
삭제 (-1.360)
네이버 (-1.373)
금고 (-1.390)
탈퇴 (-1.418)
소리 (-1.455)
최악 (-1.457)
자꾸 (-1.458)
업데이트 (

## 삼성 긍부정단어 예측

In [296]:
samsung.STAR.value_counts() #데이터불균형..

1    21316
0     1853
Name: STAR, dtype: int64

In [298]:
#데이터 스플릿 
train_texts, test_texts, train_labels, test_labels = train_test_split(samsung.token_REVIEW, samsung.STAR, test_size=0.3, random_state=123)

In [ ]:
smote = SMOTE(random_state=0)
train_texts, train_labels = smote.fit_sample(train_texts, train_labels)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트:', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

In [299]:
#TF-IDF
vectorizer = TfidfVectorizer(min_df=1, norm='l2', ngram_range=(1,1))
train_features = vectorizer.fit_transform(train_texts)
#최적모델찾기
clf = LogisticRegression(random_state=0)
params = {"C":[0.1,0.5,1,5,10]}
model= GridSearchCV(clf, param_grid=params, cv = 5, scoring='accuracy') 
model.fit(train_features, train_labels)
print('최적 하이퍼파라미터 :{0}, 정확도 :{1:.3f}'.format(model.best_params_,model.best_score_))

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regre

최적 하이퍼파라미터 :{'C': 5}, 정확도 :0.948


C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [300]:
model = LogisticRegression(C=5)

In [301]:
model.fit(train_features, train_labels)
test_features = vectorizer.transform(test_texts)
pred_labels = model.predict(test_features)
print('잘못 분류된 강의평: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

잘못 분류된 강의평: 398 out of 6951
Accuracy: 0.94


C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [302]:
coefficients = model.coef_.tolist()
sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
vocablist = [word for word, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]
#긍부정단어
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

최고 (5.925)
다만 (4.668)
매우 (3.761)
지갑 (3.544)
굿굿 (3.156)
개편 (3.116)
아주 (3.080)
신세계 (3.071)
역시 (3.067)
정말 (2.719)
아이폰 (2.648)
항상 (2.426)
출근 (2.320)
적립 (2.265)
만족 (2.255)
뱅크 (2.226)
인터넷 (2.212)
굿굿굿 (2.198)
가스 (2.182)
시스템 (2.167)
걱정 (2.135)
생활 (2.117)
편의 (2.116)
필수 (2.093)
서나 (2.007)
개꿀 (2.001)
남편 (1.984)
번만 (1.945)
기간 (1.912)
사용 (1.879)
베리 (1.877)
최고다 (1.868)
강종 (1.866)
단점 (1.864)
가장 (1.836)
하나로 (1.807)
추천 (1.800)
만사 (1.797)
점빼 (1.778)
금융 (1.766)
혁신 (1.750)
전반 (1.743)
혁명 (1.706)
별개 (1.639)
외출 (1.636)
모바일 (1.596)
조음 (1.594)
더욱 (1.593)
아우 (1.576)
소지 (1.573)
오류로 (-2.904)
점심시간 (-2.923)
방법 (-2.926)
회비 (-2.954)
별로 (-2.995)
데이터 (-3.006)
계정 (-3.021)
만기 (-3.023)
아씨 (-3.029)
초기 (-3.069)
폴드 (-3.071)
오류 (-3.089)
기본 (-3.089)
미만 (-3.104)
보지 (-3.111)
정리 (-3.122)
수준 (-3.136)
재부팅 (-3.179)
도대체 (-3.212)
통신사 (-3.238)
비밀번호 (-3.263)
광고 (-3.269)
건가 (-3.293)
전화번호 (-3.301)
금융기관 (-3.350)
준비 (-3.431)
개판 (-3.432)
며칠 (-3.479)
해제 (-3.499)
종일 (-3.521)
극협 (-3.523)
연결 (-3.547)
교통카드 (-3.553)
공지 (-3.612)
로딩 (-3.816)
어쩌 (-3.

## clustering

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1,5))
tfidf_vectorizer.fit(naver.token_REVIEW)
vector = tfidf_vectorizer.transform(text).toarray()

In [336]:
kakao

,STAR,cleaning_REVIEW,token_REVIEW
0,0,주민득록증이 없는데 어떻게 민증을 발급기간을 입력하라는 건지,주민 득록증 민증 발급 기간 입력
1,1,페이 중에는 최고인 듯,페이 최고
2,0,지문 변경한 적 없는데 지문 변경됐다고 재등록하라고 나와요 그래서 재등록 계속하는데...,지문 변경 지문 변경 등록 등록 계속 비번 입력 지문 확인 단계 무한 지문 등록 아...
3,0,비밀 최형욱 성 선정,비밀 최형욱 선정
4,0,좋으나 아쉬운 점이 페이를 충전하려면 최소금액이 만 원 이상인 것이 좀 불편하다 천...,페이 충전 최소 금액 이상인 오천 정도 최소 충전 금액
...,...,...,...
3004,0,계좌조회하는데 걸리는 시간이 분은 걸리는데 그거 기다릴 사람이 몇이나 될까요 업데이...,계좌 조회 시간 그거 사람 업데이트 메시지 짜증 점도
3005,0,써보지도 못해요 만세부터 사용 가능하네요 만세로 바꿔주시면 좋을 것 같아요 생일이 ...,보지 만세 사용 만세 생일
3006,1,진짜 엄청 좋음 카카오뱅크보다 더 좋음,진짜 카카오 뱅크
3007,0,오프라인에서 지원할 수 있는 카드 중에서 우리카드도 추가해주세요 카드 등록했더니 오...,오프라인 지원 카드 우리카드 추가 카드 등록 오프라인 미지
